In [12]:
import numpy as np
import matplotlib.pyplot as plt


class DiscreteSignal:
    def __init__(self, INF):
        self.INF=INF
        self.values= np.zeros(2*INF + 1)
        
    def set_value_at_time(self, time, value):
        if time>self.INF or time<-self.INF:
            print("Input Out Of Bound!")
        else:    
            self.values[time+self.INF]=value
            
    def shift_signal(self, shift):    
        new_signal = DiscreteSignal(self.INF) 
        new_signal.values= np.roll(self.values, shift)   
        if shift > 0:
            new_signal.values[:shift] = 0
        elif shift < 0:
            new_signal.values[shift:] = 0
        return new_signal
    
    def add(self, other):
        new_signal = DiscreteSignal(self.INF)
        new_signal.values= other.values+self.values
        return new_signal
    
    def multiply(self, other):
        new_signal = DiscreteSignal(self.INF)
        new_signal.values= other.values*self.values
        return new_signal
    
    def multiply_const_factor(self, scaler):
        new_signal = DiscreteSignal(self.INF)
        new_signal.values= scaler*self.values
        return new_signal
    
    def plot(self):
        time = np.arange(-self.INF, self.INF + 1)
    
        plt.figure(figsize=(10, 5))
    
        markerline, stemlines, baseline = plt.stem(time, self.values)
    
        plt.setp(markerline, markersize=8, color='blue')
        plt.setp(stemlines, color='blue', linewidth=1.5)
        plt.setp(baseline, visible=True)  

        plt.xlim([-self.INF - 1, self.INF + 1])
        plt.ylim([min(self.values) - 1, max(self.values) + 1])

        plt.xticks(np.arange(-self.INF, self.INF + 1, 1))
        plt.yticks(np.arange(min(self.values) - 1, max(self.values) + 1))

        plt.grid(True)

        plt.title('Discrete Signal')
        plt.xlabel('Time (n)')
        plt.ylabel('Amplitude')

        plt.show()

class DiscreteLinearTimeInvariantSystem:
    def __init__(self, impulse_response):
        self.impulse_response = impulse_response

    def linear_combination_of_impulses(self, input_signal):
        impulses = []
        coefficients = []
        for t in range(-self.impulse_response.INF, self.impulse_response.INF + 1):
            coeff = input_signal.values[t + self.impulse_response.INF]
            unit_impulse = DiscreteSignal(self.impulse_response.INF)
            unit_impulse.set_value_at_time(t, 1)
            impulses.append(unit_impulse)
            coefficients.append(coeff)
        return impulses, coefficients

    def output(self, input_signal):
        output_signal = DiscreteSignal(self.impulse_response.INF)
        impulses, coefficients = self.linear_combination_of_impulses(input_signal)
        
        for i in range(len(impulses)):
            shifted_response = self.impulse_response.shift_signal(impulses[i].values.argmax() - self.impulse_response.INF)
            scaled_response = shifted_response.multiply_const_factor(coefficients[i])
            output_signal = output_signal.add(scaled_response)
        
        return output_signal

# Stock Market Prices as a Python List
# price_list = list(map(int, input("Stock Prices: ").split()))
# n = int(input("Window size: "))
# alpha = float(input("Alpha: "))

# You may use the following input for testing purpose
price_list = [10,11,12,9,10,13,15,16,17,18]
n = 3
alpha = 0.8

INF= len(price_list)
input_Signal= DiscreteSignal(INF)
impulse_response=DiscreteSignal(INF)

for i in range(len(price_list)):
    impulse_response.set_value_at_time(i, price_list[i])
    
for i in range(n):
        input_Signal.set_value_at_time(i, alpha * ((1-alpha)**i))
        
lti_system = DiscreteLinearTimeInvariantSystem(impulse_response)
uma = lti_system.output(input_Signal).values

# Determine the values after performing Exponential Smoothing
# The length of exsm should be = len(price_list) - n + 1
exsm = []
exsm = lti_system.output(input_Signal).values
exsm= exsm[INF+n-1:]
exsm=exsm[:-(INF%n)]

print("Exponential Smoothing: " + ", ".join(f"{num:.2f}" for num in exsm))
# Output should be: 11.68, 9.47, 9.82, 12.29, 14.40, 15.62, 16.64, 17.63

Exponential Smoothing: 11.68, 9.47, 9.82, 12.29, 14.40, 15.62, 16.64, 17.63
